## RAG 시나리오(문서)
이 데모는 RAG와 RAG 없는 경우를 비교하는 데모임.
LLM 서비스는 로컬 Ollama 서비스, LLAMA3.2 2b와 lanchain 프롬프트를 사용합니다. 
1. 시나리오1: LLM에 직접 질문&답.
2. 시나리오2(RAG): DB 데이터의 벡터검색 결과를 LLM이 참조하는 답변.
3. 2개의 시나리오 결과를 비교해보세요 

### 0. 파이썬 라이브러리 로딩 및 오라클 DB 접속

In [1]:
# 파이썬 라이브러리 로딩 및 오라클 DB 접속
import oracledb,oml
import re, os, time
import pandas as pd
import import_ipynb
from IPython.display import display
from user_functions import get_conn_cursor,get_db_version,exec_sql,exec_sql_rows,dml_sql,ddl_sql,clean_text

# DB 접속
user_id = 'labadmin'
pwd = 'labadmin'
dbconn = 'dbserver26ai:1521/freepdb1'

conn, cursor = get_conn_cursor(user_id,pwd,dbconn)

# DB 버전 확인
get_db_version(cursor)

/opt/jupyterhub/vector_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,PRODUCT,VERSION_FULL
0,Oracle AI Database 26ai Free,23.26.0.0.0


### 1. RAG DB 데이터 준비
1. 문서(소프트웨어분리발주메뉴얼)->텍스트 추출->청킹->임베딩 실행
2. 임베딩 정보를 테이블(rag_tbl_v)에 저장, 벡터 인덱싱 생성

### 참고 : 시나리오에 사용할 문서

In [7]:
# pdf 문서(소프트웨어 분리발주 매뉴얼)디스플레이
import os
from IPython.display import FileLink
img_dir = "./data/doc/rag/"  
img_file = "소프트웨어_분리발주_매뉴얼_개정배포_16.7.pdf" 
img_path = os.path.join(img_dir, img_file)

display(FileLink(img_path))

/home/labadmin/labs/vector/similarity_search/data/doc/rag/소프트웨어_분리발주_매뉴얼_개정배포_16.7.pdf

### 임베딩 및 인덱싱
문서 테이블(rag_tbl), 청킹 및 벡터 저장 테이블(rag_tbl_v), 인덱싱(HNSW)

In [2]:
# 벡터 테이블 정리
dr_tbl = """truncate table rag_tbl_v """
ddl_sql(cursor,dr_tbl)
print("테이블이 정리 됐습니다")

# 테이블 저장 문서에서 텍스트 추출 및 청킹
# 임베딩 및 인덱싱
print("임베딩 및 인덱싱을 진행중입니다 ......")
cr_tbl = """
insert into rag_tbl_v  
SELECT dt.id AS 문서번호, et.embed_id as 청크ID , et.embed_data AS 문서내용, 
       to_vector(et.embed_vector) as 벡터
  FROM rag_tbl dt, 
         dbms_vector_chain.utl_to_embeddings(
         dbms_vector_chain.utl_to_chunks(
           dbms_vector_chain.utl_to_text(dt.doc),
           JSON('{"by" : "words",
                  "max" : "100", 
                  "overlap" : "20",
                  "split" : "newline",
                  "language" : "korean",
                  "normalize" : "all"}')),
           JSON('{"provider":"database", 
                  "model":"MULTILINGUAL_E5_SMALL"}')) t,
       JSON_TABLE(t.column_value, 
                  '$'columns (embed_id number,embed_data varchar2(4000), 
                  embed_vector CLOB)) et
WHERE dt.id = 3
"""
start_time = time.time()
rows = ddl_sql(cursor,cr_tbl)
execute_time = time.time() - start_time
print("임베딩 완료 시간 :",execute_time)

# 벡터 인덱싱 정리
ddl_sql(cursor,"drop index if exists rag_tbl_v_idx")

# 벡터 인덱싱
crIdx = """ create vector index rag_tbl_v_idx on rag_tbl_v(벡터)
ORGANIZATION INMEMORY NEIGHBOR GRAPH WITH DISTANCE COSINE
"""
ddl_sql(cursor,crIdx)
print("인덱싱 완료")
print("벡터 데이터 준비가 됐습니다.")

테이블이 정리 됐습니다
임베딩 및 인덱싱을 진행중입니다 ......
임베딩 완료 시간 : 31.806585550308228
인덱싱 완료
벡터 데이터 준비가 됐습니다.


### 공통질문

In [10]:
# 공통 질문
user_q = "공공기관 분리발주 대상 소프트웨어들은 어떤 것들이 있나요?"
print("공통질문 :",user_q)

공통질문 : 공공기관 분리발주 대상 소프트웨어들은 어떤 것들이 있나요?


### 2. LLM 질의 답변 경우
- LLM : LLAMA3.2
- LLM에 보강 프롬프트없이 질문합니다.

In [11]:
# LLM(llama3.2) 질의 응답
# Oracle's API for ollama
gen_param = """{"provider":"ollama", "host":"local", "url":"http://service-ollama:11434/api/generate", "model":"llama3.2"}"""

gen_q = """select dbms_vector.utl_to_generate_text(:txt, json(:param)) from dual"""
print("LLAMA3.2에 질문 : ",user_q," 에 대한 답변을 준비중입니다......")

import time
start_time = time.time()
result = exec_sql_rows(cursor,gen_q,{"param":gen_param,"txt":user_q})
execute_time = time.time() - start_time
lob_result = result[0][0]
print("LLAMA3.2가 학습된 정보를 기초로 답변한 내용은 다음과 같습니다.")
print("---------------------------------------------------------------------")
print(lob_result.read())
print("---------------------------------------------------------------------")
print("실행 시간 :",execute_time)

LLAMA3.2에 질문 :  공공기관 분리발주 대상 소프트웨어들은 어떤 것들이 있나요?  에 대한 답변을 준비중입니다......
LLAMA3.2가 학습된 정보를 기초로 답변한 내용은 다음과 같습니다.
---------------------------------------------------------------------
공공기관 분리발주 대상 소프트웨어는 공공기관이 필요하는 서비스를 제공할 수 있는 소프트웨어입니다. 다음과 같은 예가 있습니다.

1. 의료 정보 시스템 (EHR): 의사당에서 사용되는 의학信息시스템을 개발하기 위해 필요한 소프트웨어입니다.
2. 자치구 정보시스템: 지방자치단결형의 자치구가 필요하는 정보시스템을 개발하기 위해 필요한 소프트웨어입니다.
3. 통신망 시스템: 공공기관이 필요한 통신망 시스템을 개발하기 위해 필요한 소프트웨어입니다.
4. 인사 관리 시스템 (HRMS): 공공기관에서 사용되는 인사 관리 시스템을 개발하기 위해 필요한 소프트WARE입니다.
5. مالی información 시스템: 공공기관이 nécess하는 finanzi적인 정보를 수집하고 관리하기위한 시스템을 개발하기 위해 필요한 소프트웨어입니다.

위의 예는 공공기관 분리발주 대상 소프트웨어의 일반적인 예입니다. 공공기관은 다양한need에 따라 소프트웨어를 필요로 할 수 있으므로, 이 목록은 완전하지 않습니다.
---------------------------------------------------------------------
실행 시간 : 18.47319483757019


### 3. RAG 실행 경우
- RAG 실행 순서는 1. 사용자 질의가 들어 있는 문장에 대한 DB 벡터 검색, 2. 벡터 검색 결과를 참조하는 프롬프트 생성, 3. 프로프트를 LLM에 전달, 답변 요청, 4. LLM 답변
- LLM : LLAMA3.2

In [13]:
# 사용자 지정 문서를 참조한 RAG 답변

from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

print(user_q,"에 대한 답변을 DB문서를 참조해서 답변을 준비중입니다......")

# 1. vector search
sim_search = """
select T.문서번호,T.청크ID, T.문서내용
from rag_tbl_v T
order by vector_distance(T.벡터,
                         dbms_vector.utl_to_embedding(:u_sentence, 
                                     json('{"provider":"database", 
                                            "model":"MULTILINGUAL_E5_SMALL"}')), cosine)
fetch first 3 rows only
"""
simq_start_time = time.time()
sim_result = exec_sql(cursor,sim_search,{"u_sentence":user_q})
sim_result = clean_text(sim_result)
sim_result['문서내용']=sim_result['문서내용'].str.replace('�', '', regex=False)
simq_time = time.time() - simq_start_time

print("벡터쿼리 실행 시간 ",simq_time)

# ollama API by oracle
gen_ollama_params = """{"provider":"ollama", 
                          "host":"local", 
                          "url":"http://service-ollama:11434/api/generate",
                          "model":"llama3.2"}
                          """
user_question = user_q

## 2. 프롬프트 생성 
template = """
Context 정보만를 이용해서 답변해주세요.
'답변'과'참조문서'로 구분해서 답변해주세요.답변 작성시 친숙한 어조로 작성해주세요.
'답변'은 500자 이내로 요약 정리해주세요. '참조문서'에는 context의 '문서명'과 '청크ID'를 명시해주세요.
CONTEXT : {context}
[INST]QUESTION : {question}[/INST]
"""
print("프롬프트 \n", template)

prompt = ChatPromptTemplate.from_template(template)
print("langchain-ollama 프롬프트를 생성하였고, LLAMA3.2가 답변을 생성중입니다.......")

# 3. LLM 답변 생성
model = OllamaLLM(model="llama3.2", max_new_tokens= 2000 ,temperature=0.1, base_url='http://service-ollama:11434')
chain = prompt | model 

rag_start_time = time.time()
answer = chain.invoke({"context":sim_result,"question": user_question})
rag_time = time.time() - rag_start_time

print("LLM 답변 생성시간 : ",rag_time)
print("-----------------------------------------------------------------------------------")
print(answer)
print("-----------------------------------------------------------------------------------")

공공기관 분리발주 대상 소프트웨어들은 어떤 것들이 있나요? 에 대한 답변을 DB문서를 참조해서 답변을 준비중입니다......
벡터쿼리 실행 시간  0.01801013946533203
프롬프트 
 
Context 정보만를 이용해서 답변해주세요.
'답변'과'참조문서'로 구분해서 답변해주세요.답변 작성시 친숙한 어조로 작성해주세요.
'답변'은 500자 이내로 요약 정리해주세요. '참조문서'에는 context의 '문서명'과 '청크ID'를 명시해주세요.
CONTEXT : {context}
[INST]QUESTION : {question}[/INST]

langchain-ollama 프롬프트를 생성하였고, LLAMA3.2가 답변을 생성중입니다.......
LLM 답변 생성시간 :  23.97117805480957
-----------------------------------------------------------------------------------
**답변**

분리발주 대상 소프트웨어는 공공기관이 필요로 하는 소프트웨어를 의미합니다. 이 중에서 공공기관이 필요로 하는 소프트웨어의 종류는 다음과 같습니다.

1.  **인터넷 및 네트워크 소프트웨어**: 공공기관은 인터넷과 네트워크를 사용하여 업무를 수행해야 합니다. 따라서 인터넷 및 네트워크 소프트웨어가 분리발주 대상 소프트웨어 중 하나입니다.
2.  **데이터 분석 및 처리 소프트웨어**: 공공기관은 데이터를 수집, 분석, 처리해야 합니다. 따라서 데이터 분석 및 처리 소프트웨어가 분리발주 대상 소프트WARE 중 하나입니다.
3.  **보안 소프트웨어**: 공공기관은 보안을 중요하게 생각해야 합니다. 따라서 보안 소프트웨어가 분리발주 대상 소프트웨어 중 하나입니다.
4.  **소프트웨어 개발 및 유지 관리 소프트웨어**: 공공기관은 소프트웨어를 개발, 유지 관리해야 합니다. 따라서 소프트웨어 개발 및 유지 관리 소프트웨어가 분리발주 대상 소프트웨어 중 하나입니다.

이러한 소프트웨어는 공공기관의 업무에 필

### 참고 : 벡터 검색 결과

In [14]:
# 벡터 검색 결과
display(
    sim_result.head(10).style
    .set_properties(**{'text-align': 'left'})
        .set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])
)

,문서번호,청크ID,문서내용
0,3,176,"▣ 「분리발주 대상 소프트웨어」(미래창조과학부 고시) 제 3조(분리발주 대상 소프트웨어) ⑤국가,지방자치단체,국가또는지방자치단체가투자하거나출연한법인또는 그밖의공공단체등(이하""국가기관등""이라 한다)의장은분리발주대상소프트웨어가제1항제1호에따른국가종합"
1,3,176,"▣ 「분리발주 대상 소프트웨어」(미래창조과학부 고시) 제 3조(분리발주 대상 소프트웨어) ⑤국가,지방자치단체,국가또는지방자치단체가투자하거나출연한법인또는 그밖의공공단체등(이하""국가기관등""이라 한다)의장은분리발주대상소프트웨어가제1항제1호에따른국가종합"
2,3,296,"미래창조과학부고시 제2015-107호 「소프트웨어산업 진흥법」제20조제2항에 따른 「분리발주 대상 소프트웨어(미래창조과학부고시 제2014-80호)」전부를 개정하고, 다음과 같이 고시합니다. 2015년 12월 28일 미래창조과학부장관 분리발주 대상 소프트웨어 제 1조(목적) 이 고시는 소프트웨어산업 진흥법 제20조제2항에 따라 국가, 지방자치단체, 국가 또는 지방 자치단체가 투자하거나 출연한 법인 또는 그 밖의 공공단체 등의 장이 소프트웨어를 분리발주하는 경우 세부적인 대상 및 방법 등을 정하는 것을 목적으로 한다. 제 2조(분리발주 대상 사업)"


시나리오 종료

--------------------------